In [3]:
import sys

sys.path.append(r'C:\repos\ra2ce_multi_network') # path to the package
sys.path.append(r'C:\repos\ra2ce') # path to the package

import geopandas as gpd
import pickle

import pyproj
from networkx import MultiDiGraph
from pathlib import Path
from snkit import Network

from ra2ce.graph.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter
from ra2ce.graph.exporters.network_exporter_base import NetworkExporterBase
from ra2ce.graph.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.graph.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper
from ra2ce.graph.origins_destinations import add_od_nodes
from ra2ce_multi_network.simplify_rail import _nx_to_network, _network_to_nx
import ra2ce.graph.networks_utils as nut

In [4]:
# Defining ini variables
root_folder = Path(
    r'C:\Users\asgarpou\OneDrive - Stichting Deltares\Documents\Projects\Long involvement\SITO_Netwerk to system\project_files\get_shortest_routes_road')
study_area_suffix = '_GR'
output_name = f'study_area{study_area_suffix}'
od_file = root_folder.joinpath('static/network/od_nodes.geojson')

In [5]:
# Load complex graph
od_gdf = gpd.read_file(od_file)

with open(root_folder.joinpath(f'static/output_graph/road_complex_graph_{output_name}.p'), 'rb') as f:
    complex_graph = pickle.load(f)

KeyboardInterrupt: 

In [4]:
# Simplify complex graph => get_network procedure source == "OSM download"

simple_graph, complex_graph, link_tables = nut.create_simplified_graph(complex_graph)

100%|██████████| 161369/161369 [00:00<00:00, 392805.76it/s]
ERROR:root:Could not find the simple ID belonging to complex ID (30011, 32534, 0); value set to None. Full error: 61908
ERROR:root:Could not find the simple ID belonging to complex ID (30011, 32537, 0); value set to None. Full error: 61909
ERROR:root:Could not find the simple ID belonging to complex ID (32532, 32535, 0); value set to None. Full error: 66639
ERROR:root:Could not find the simple ID belonging to complex ID (32532, 32533, 0); value set to None. Full error: 66640
ERROR:root:Could not find the simple ID belonging to complex ID (32533, 32532, 0); value set to None. Full error: 66641
ERROR:root:Could not find the simple ID belonging to complex ID (32533, 32534, 0); value set to None. Full error: 66642
ERROR:root:Could not find the simple ID belonging to complex ID (32534, 32533, 0); value set to None. Full error: 66643
ERROR:root:Could not find the simple ID belonging to complex ID (32534, 30011, 0); value set to None

In [6]:
# Save the link tables linking complex and simple IDs
osm_network_wrapper = OsmNetworkWrapper(config_data=NetworkConfigData())
# osm_network_wrapper._export_linking_tables(link_tables)

if not osm_network_wrapper.is_directed and isinstance(simple_graph, MultiDiGraph):
    simple_graph = simple_graph.to_undirected()

# Check if all geometries between nodes are there, if not, add them as a straight line.

simple_graph = nut.add_missing_geoms_graph(simple_graph, geom_name="geometry")
simple_graph.graph["crs"] = pyproj.CRS("EPSG:4326")

In [7]:
# Save the outputs

od_gdf.to_file(od_file, driver="GeoJSON")

simple_graph_exporter = MultiGraphNetworkExporter(
    basename=f'road_simple_graph_{output_name}',
    export_types=['pickle', 'shp'])
simple_graph_exporter.export_to_shp(output_dir=root_folder.joinpath(f'static/output_graph'),
                                    export_data=simple_graph)
simple_graph_exporter.export_to_pickle(output_dir=root_folder.joinpath(f'static/output_graph'),
                                       export_data=simple_graph)

In [14]:
# Load graphs and networks prepared befor

od_gdf = gpd.read_file(od_file)

# Approach 1
with open(root_folder.joinpath(f'static/output_graph/road_simple_graph_{output_name}.p'), 'rb') as f:
    simple_graph = pickle.load(f)

simple_road_network = Network(
        nodes=_nx_to_network(simple_graph).nodes,
        edges=_nx_to_network(simple_graph).edges
    )

# Approach 2
# simple_road_network = Network(
#     nodes=gpd.read_file(root_folder.joinpath(f'static/output_graph/road_simple_graph_nodes_{output_name}.gpkg')),
#     edges=gpd.read_file(root_folder.joinpath(f'static/output_graph/road_simple_graph_edges_{output_name}.gpkg'))
# )

In [8]:
# Map origin-destinations the simple graphs

road_simple_graph_od_mapped = add_od_nodes(od=gpd.read_file(od_file), graph=simple_graph, crs=pyproj.CRS("EPSG:4326"))[1]

Adding Origin-Destination nodes to graph: 0it [00:00, ?it/s]C:\repos\ra2ce\ra2ce\graph\origins_destinations.py:187: ShapelyDeprecationWarning: The 'almost_equals()' method is deprecated and will be removed in Shapely 2.1; use 'equals_exact()' instead
  if Point(graph.nodes[node_a]["geometry"].coords[0]).almost_equals(
C:\repos\ra2ce\ra2ce\graph\origins_destinations.py:202: ShapelyDeprecationWarning: The 'almost_equals()' method is deprecated and will be removed in Shapely 2.1; use 'equals_exact()' instead
  if Point(graph.nodes[node_b]["geometry"].coords[0]).almost_equals(
C:\repos\ra2ce\ra2ce\graph\origins_destinations.py:217: ShapelyDeprecationWarning: The 'almost_equals()' method is deprecated and will be removed in Shapely 2.1; use 'equals_exact()' instead
  if Point(graph.nodes[node_a]["geometry"].coords[0]).almost_equals(
C:\repos\ra2ce\ra2ce\graph\origins_destinations.py:232: ShapelyDeprecationWarning: The 'almost_equals()' method is deprecated and will be removed in Shapely 2.1

In [9]:
# Save the outputs

od_gdf.to_file(od_file, driver="GeoJSON")

simple_graph_od_mapped_exporter = MultiGraphNetworkExporter(
    basename=f'road_simple_graph_od_mapped_{output_name}',
    export_types=['pickle', 'shp'])
simple_graph_od_mapped_exporter.export_to_shp(output_dir=root_folder.joinpath(f'static/output_graph'),
                                    export_data=road_simple_graph_od_mapped)
simple_graph_od_mapped_exporter.export_to_pickle(output_dir=root_folder.joinpath(f'static/output_graph'),
                                       export_data=road_simple_graph_od_mapped)

In [13]:
# Visualise

road_map = road_simple_graph_od_mapped.edges.explore(tiles="CartoDB positron")
road_map = od_gdf.explore(m=road_map, color='springgreen')
road_simple_graph_od_mapped.nodes[~road_simple_graph_od_mapped.nodes['od_id'].isna()].explore(m=road_map, color='darkgreen')